<a href="https://colab.research.google.com/github/Neil-Cardoz/Deep-Learning/blob/main/DLL_LAB_RNN_ARCHITECTURE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np



In [ ]:
class RNN:
  def __init__(self, input_size, output_size, hidden_size=64):
    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    # Innitialize Weights
    self.Wxh = np.random.randn(hidden_size, hidden_size) * 0.01
    self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
    self.Why = np.random.randn(hidden_size, hidden_size) * 0.01

    # Innitalize Biases
    self.bh = np.zeros((1, hidden_size))
    self.by = np.zeros((1, hidden_size))


  def forward(self, inputs):
    h_prev = np.zeros((self.hidden_size, 1))
    self.last_inputs = inputs
    self.last_hs = {0: h_prev}


    # Forward pass
    for i, x in enumerate(inputs):
      x = np.reshape(x, (self.input_size, 1))
      h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h_prev) + self.bh)
      self.last_hs[i+1] = h
      h_prev = h

    # Compute the output
    output = np.dot(self.Why, h_prev) + self.by
    return output, h_prev

def backward(self, learning_rate=0.01):
  n = len(self.last_inputs)
  d_Why = np.zeros_like(self.Why)
  d_Whh = np.zeros_like(self.Whh)
  d_Wxh = np.zeros_like(self.Wxh)
  d_bh = np.zeros_like(self.bh)
  d_by = np.zeros_like(self.by)
  d_h = np.dot(self.Why.T, d_y)

  for t in reversed(range(n)):
    temp = self.last_hs[t+1]
    d_Why += np.dot(d_y, temp.T)
    d_h_raw = (1 - self.last_hs[t+1] ** 2) * d_h

    d_bh += d_h_raw
    d_Wxh += np.dot(d_h_raw, self.last_inputs[t].T.reshape(-1, 1))
    d_Whh += np.dot(d_h_raw, self.last_hs[t].T)
    d_h = np.dot(self.Wxh.T, d_h_raw)


    # Update Weights and Biases
    self.Why -= learning_rate * d_Why
    self.Wxh -= learning_rate * d_Wxh
    self.Whh -= learning_rate * d_Whh
    self.bh -= learning_rate * d_bh
    self.by -= learning_rate * d_by


# Example sequence (One hot encoded input)
input = [np.array([1,0,0]), np.array([0,1,0]), np.array([0,0,1])]
rnn = RNN(input_size=3, hidden_size=3, output_size=2)

# Forward Pass
output, hidden_state = rnn.forward(input)
print(output)

# Backward Pass
d_y = np.array([[0],[1]])
rnn.backward(d_y)

